This is an example of mmdetection using Faster RCNN,you can change the configuration file code and the weight path to fit your own model. 

Reference:https://www.kaggle.com/mlneo07/mmdetection-swin-transfomer-frcnn-inference-0-443

# **Install libraries**

In [ ]:
!pip install '../input/faster-rcnn-mmdetection/fcnn_dep_0/fcnn_dep_0/addict-2.4.0-py3-none-any.whl'
!pip install '../input/faster-rcnn-mmdetection/fcnn_dep_0/fcnn_dep_0/asttokens-2.0.5-py2.py3-none-any.whl'
!pip install '../input/faster-rcnn-mmdetection/fcnn_dep_0/fcnn_dep_0/executing-0.8.2-py2.py3-none-any.whl'
!pip install '../input/faster-rcnn-mmdetection/fcnn_dep_0/fcnn_dep_0/yapf-0.32.0-py2.py3-none-any.whl'
!pip install '../input/faster-rcnn-mmdetection/faster_rcnn_mmdetection/fcnn_dep/torch-1.7.0+cu110-cp37-cp37m-linux_x86_64.whl'
#!pip install '../input/faster-rcnn-mmdetection/faster_rcnn_mmdetection/fcnn_dep/torchvision-0.8.1+cu110-cp37-cp37m-linux_x86_64.whl'
!pip install '../input/faster-rcnn-mmdetection/faster_rcnn_mmdetection/fcnn_dep/terminaltables-3.1.10-py2.py3-none-any.whl' 
#!pip install '../input/faster-rcnn-mmdetection/faster_rcnn_mmdetection/fcnn_dep/setuptools-46.4.0-py3-none-any.whl' 
#!pip install '../input/faster-rcnn-mmdetection/faster_rcnn_mmdetection/fcnn_dep/numpy-1.16.0-cp37-cp37m-manylinux1_x86_64.whl'
!pip install '../input/faster-rcnn-mmdetection/faster_rcnn_mmdetection/fcnn_dep/mmcv_full-1.3.9-cp37-cp37m-manylinux1_x86_64.whl'
!pip install '../input/faster-rcnn-mmdetection/faster_rcnn_mmdetection/fcnn_dep/imagecorruptions-1.1.2-py3-none-any.whl'
!pip install '../input/faster-rcnn-mmdetection/faster_rcnn_mmdetection/fcnn_dep/icecream-2.0.0-py2.py3-none-any.whl'
!pip install '../input/faster-rcnn-mmdetection/faster_rcnn_mmdetection/fcnn_dep/ensemble_boxes-1.0.8-py3-none-any.whl'
!pip install '../input/faster-rcnn-mmdetection/faster_rcnn_mmdetection/fcnn_dep/albumentations-0.5.0-py3-none-any.whl'
#!pip install '../input/faster-rcnn-mmdetection/faster_rcnn_mmdetection/fcnn_dep/Pillow-6.2.2-cp37-cp37m-manylinux1_x86_64.whl'
#!pip install '../input/faster-rcnn-mmdetection/faster_rcnn_mmdetection/fcnn_dep/Cython-0.29.15-cp37-cp37m-manylinux1_x86_64.whl'
!pip install '../input/faster-rcnn-mmdetection/faster_rcnn_mmdetection/fcnn_dep/mmpycocotools-12.0.3'

In [ ]:
!cp -r ../input/faster-rcnn-mmdetection/faster_rcnn_mmdetection/kaggle_mmdetection /kaggle/working/
%cd /kaggle/working/kaggle_mmdetection
!pip install -e .
%cd ..

# **Import libraries**

In [ ]:
import sys, os
sys.path.append(r'./kaggle_mmdetection')
sys.path.append(r'../input/tensorflow-great-barrier-reef/greatbarrierreef')
import numpy as np
import mmdet
import mmcv
from mmdet.datasets.builder import build_dataset
from mmdet.models.builder import build_detector
from mmdet.apis.train import train_detector,set_random_seed
from mmdet.apis.inference import inference_detector, init_detector, show_result_pyplot

# **Model**

In [ ]:


%%writefile /kaggle/working/kaggle_mmdetection/output/faster_rcnn/faster_rcnn_r50_fpn_1x_coco_starfish.py


fp16 = dict(loss_scale=512.)  #混合精度训练
# model settings
model = dict(
    type='FasterRCNN',  #检测算法
    pretrained=None,  # 预训练模型,None:自己加载，不选择在线下载
    backbone=dict(
        type='ResNet',  #骨干网
        depth=50,       #层数：50,101,152
        num_stages=4,   #resnet的stage数量
        out_indices=(0, 1, 2, 3), # 输出的stage的序号
        frozen_stages=1, #冻结第一个stage, 即该stage不更新参数
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        #init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    ),
    neck=dict(
        type='FPN',  # neck类型
        in_channels=[256, 512, 1024, 2048],  # 输入的各个stage的通道数
        out_channels=256,  # 输出的特征层的通道数
        num_outs=5),     # 输出的特征层的数量
    rpn_head=dict(
        type='RPNHead',    # RPN网络类型
        in_channels=256,   # RPN网络的输入通道数
        feat_channels=256, # 特征层的通道数
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],    # 生成的anchor的baselen，baselen = sqrt(w*h)，w和h为anchor的宽和高
            ratios=[0.5, 1.0, 2.0],  # anchor的宽高比
            strides=[4, 8, 16, 32, 64]),  # 在每个特征层上的anchor的步长（对应于原图）
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[.0, .0, .0, .0],  # 均值
            target_stds=[1.0, 1.0, 1.0, 1.0]),  # 方差
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),   
            # 是否使用sigmoid来进行分类，如果False则使用softmax来分类
        loss_bbox=dict(type='L1Loss', loss_weight=1.0)),
    roi_head=dict(
        type='StandardRoIHead',  # RoIExtractor类型
        bbox_roi_extractor=dict(
            type='SingleRoIExtractor',
            roi_layer=dict(type='RoIAlign', output_size=7, sampling_ratio=0),  
            # ROI具体参数：ROI类型为ROIalign，输出尺寸为7，sample数为2
            out_channels=256,    # 输出通道数
            featmap_strides=[4, 8, 16, 32]),   # 特征图的步长
        bbox_head=dict(
            type='Shared2FCBBoxHead',   # 全连接层类型
            in_channels=256,             # 全连接层数量
            fc_out_channels=1024,       # 输入通道数
            roi_feat_size=7,            # 输出通道数
            #num_classes=12,             # 分类器的类别数量
            num_classes=1,             # 分类器的类别数量
            bbox_coder=dict(
                type='DeltaXYWHBBoxCoder',
                target_means=[0., 0., 0., 0.],
                target_stds=[0.1, 0.1, 0.2, 0.2]),
            reg_class_agnostic=False,    
            # 是否采用class_agnostic的方式来预测，class_agnostic表示输出bbox时只考虑其是否为前景，
            # 后续分类的时候再根据该bbox在网络中的类别得分来分类，也就是说一个框可以对应多个类别
            loss_cls=dict(
                type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1.0),
            loss_bbox=dict(type='L1Loss', loss_weight=1.0))))

# model training and testing settings
train_cfg=dict(
    rpn=dict(
        assigner=dict(
            type='MaxIoUAssigner',  # RPN网络的正负样本划分
            pos_iou_thr=0.7,        # 正样本的iou阈值
            neg_iou_thr=0.3,        # 负样本的iou阈值
            min_pos_iou=0.3,        # 正样本的iou最小值。
            #如果assign给ground truth的anchors中最大的IOU低于0.3，则忽略所有的anchors，否则保留最大IOU的anchor
            match_low_quality=True,
            ignore_iof_thr=-1),     # 忽略bbox的阈值，当ground truth中包含需要忽略的bbox时使用，-1表示不忽略
        sampler=dict(
            type='RandomSampler',   # 正负样本提取器类型
            num=256,                # 需提取的正负样本数量
            pos_fraction=0.5,        # 正样本比例
            neg_pos_ub=-1,          # 最大负样本比例，大于该比例的负样本忽略，-1表示不忽略
            add_gt_as_proposals=False),  # 把ground truth加入proposal作为正样本
        allowed_border=-1,          # 不允许在bbox周围外扩一定的像素, 0允许
        pos_weight=-1,              # 正样本权重，-1表示不改变原始的权重
        debug=False),               
    rpn_proposal=dict(
        nms_pre=2000,               # 在nms之前保留的的得分最高的proposal数量
        max_per_img=1000,           # 在nms之后保留的的得分最高的proposal数量
        nms=dict(type='nms', iou_threshold=0.7), # nms阈值
        min_bbox_size=0),           # 最小bbox尺寸
    rcnn=dict(
        assigner=dict(
            type='MaxIoUAssigner',    # RCNN网络正负样本划分
            pos_iou_thr=0.3,           # 正样本的iou阈值
            neg_iou_thr=0.3,           # 负样本的iou阈值
            min_pos_iou=0.3,           
# 正样本的iou最小值。如果assign给ground truth的anchors中最大的IOU低于0.3，则忽略所有的anchors，否则保留最大IOU的anchor
            match_low_quality=False,
            ignore_iof_thr=-1),        # 忽略bbox的阈值，当ground truth中包含需要忽略的bbox时使用，-1表示不忽略
        sampler=dict(
            type='RandomSampler',      # 正负样本提取器类型
            num=512,                   # 需提取的正负样本数量
            pos_fraction=0.25,         # 正样本比例
            neg_pos_ub=-1,             # 最大负样本比例，大于该比例的负样本忽略，-1表示不忽略
            add_gt_as_proposals=True), # 把ground truth加入proposal作为正样本
        pos_weight=-1,                 # 正样本权重，-1表示不改变原始的权重
        debug=False))

test_cfg=dict(
    rpn=dict(
        nms_pre=1000,
        max_per_img=1000,
        nms=dict(type='nms', iou_threshold=0.7),
        min_bbox_size=0),
    rcnn=dict(
        score_thr=0.05,
        nms=dict(type='nms', iou_threshold=0.5),
        max_per_img=100)
)

# data setting
dataset_type = 'StarfishDataset'   # 数据集类型
#data_root = '/cache/kaggle_dataset/'  # 数据集根目录
data_root = '/kaggle/working/kaggle_mmdetection/kaggle_dataset/'  # 数据集根目录
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True) 
    # 输入图像初始化，减去均值mean并处以方差std，to_rgb表示将bgr转为rgb
train_pipeline = [
dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    #dict(type='MixUp',p=0.5, lambd=0.5),   #图像混合
    dict(type='Resize', img_scale=[(4096, 600), (4096, 1000)],
         multiscale_mode='range', keep_ratio=True),
    #dict(type='Rotate', prob=0.5,level=10, max_rotate_angle=30), #图像旋转 30
    dict(type='RandomFlip', direction=['horizontal'], flip_ratio=0.5),
    #dict(type='BBoxJitter', min=0.95, max=1.05),   #框抖动
    dict(type='Normalize', **img_norm_cfg),
    dict(type='Pad', size_divisor=32),
    #dict(type='Grid', use_w=True, use_h=True),  #掩码遮挡
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels']),
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=[(4096, 600),(4096, 800),(4096, 1000)], # 输入图像尺寸
        flip=True,  # 是否翻转
        transforms=[
            dict(type='Resize', keep_ratio=True),   
            dict(type='RandomFlip'), 
            dict(type='Normalize', **img_norm_cfg),
            dict(type='Pad', size_divisor=32),  
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]
data = dict(
    samples_per_gpu=1,   # 每个gpu计算的图像数量
    workers_per_gpu=1,   # 每个gpu分配的线程数
    train=dict(
        type=dataset_type,
        ann_file=data_root + 'annotations/train.json',  # 数据集annotation路径
        img_prefix=data_root + 'train2017',    # 数据集annotation路径
        pipeline=train_pipeline),
    val=dict(
        type=dataset_type,
        #ann_file=data_root + 'annotations/instances_train2017.json',
        ann_file=data_root + 'annotations/valid.json',
        img_prefix=data_root + 'val2017',
        pipeline=test_pipeline),
    test=dict(
        type=dataset_type,
        ann_file='cache/test.json',
        img_prefix='cache/test',
        pipeline=test_pipeline))

# optimizer
optimizer = dict(type='SGD',lr=0.00125, momentum=0.9, weight_decay=0.0001)
optimizer_config = dict(grad_clip=None)
# learning policy
lr_config = dict(policy='step', warmup='linear', warmup_iters=500,  
warmup_ratio=1.0 / 3, step=[8, 11])   
# 优化策略, 初始的学习率增加的策略，linear为线性增加
# 在初始的500次迭代中学习率逐渐增加
# 起始的学习率,在第8和11个epoch时降低学习率
log_config = dict(interval=100,   # 每100个batch输出一次信息
    hooks=[
        dict(type='TextLoggerHook'),   # 控制台输出信息的风格
    ])
custom_hooks = [dict(type='NumClassCheckHook')] 
dist_params = dict(backend='nccl')
log_level = 'INFO'
checkpoint_config = dict(interval=1) # 每1个epoch存储一次模型
#evaluation = dict(interval=4, metric='bbox') # 每4个epoch测试一次
evaluation = dict(interval=2, metric='bbox') # 每2个epoch测试一次
#runner = dict(type='EpochBasedRunner', max_epochs=12)  # 最大epoch数
runner = dict(type='EpochBasedRunner', max_epochs=30)  # 最大epoch数
# runtime
#load_from = r'/cache/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth'
load_from = r'/kaggle/working/pre_model/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth'
# 加载模型的路径，None表示从预训练模型加载
#work_dir = r'/cache/log/'  # log文件和模型文件存储路径
work_dir = r'/kaggle/working/kaggle_mmdetection/output/faster_rcnn'  # log文件和模型文件存储路径
resume_from = None    # 恢复训练模型的路径
workflow = [('train', 1)]   # 当前工作区名称


In [ ]:
from mmcv import Config
cfg = Config.fromfile('/kaggle/working/kaggle_mmdetection/output/faster_rcnn/faster_rcnn_r50_fpn_1x_coco_starfish.py')

In [ ]:
%%writefile labels.txt 
starfish

# **Inference**

In [ ]:
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))

    return " ".join(pred_strings)

In [ ]:
import greatbarrierreef

env = greatbarrierreef.make_env()   # initialize the environment
iter_test = env.iter_test()  

In [ ]:
model = init_detector(cfg, '/kaggle/input/faster-rcnn-mmdetection/faster_rcnn_mmdetection/fcnn_pth/epoch_4.pth')

In [ ]:
#################################### faster rcnn ############################################
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# model = build_detector(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)
# checkpoint = load_checkpoint(model, WEIGHTS_FILE, map_location='cpu')

# model.CLASSES = dataset.CLASSES

# model = MMDataParallel(model, device_ids=[0])
# outputs = single_gpu_test(model, data_loader, False, None, 0.5)

results = []

for (pixel_array, sample_prediction_df) in iter_test:
    result = inference_detector(model, pixel_array[:, :, ::-1])
#     show_result_pyplot(model, pixel_array[:, :, ::-1], result)    
    boxes = result[0][:, :4]
    scores = result[0][:, 4]

    boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
    boxes[:, 3] = boxes[:, 3] - boxes[:, 1]

    
    sample_prediction_df['annotations'] = format_prediction_string(boxes, scores)
    
    env.predict(sample_prediction_df)

In [ ]:
%rm -rf kaggle_mmdetection
%rm labels.txt